In [ ]:
import numpy as np
import scipy.io as sio

def save_as_mat(matrix, filename):
    sio.savemat(filename, {'adjacency_matrix': matrix})

N = 2000
alpha = np.array([0.75])  # alpha 现在表示偏好连边的比例
degree_interpolate = np.zeros((2, len(alpha), N))
m0 = 3  # 初始节点数

for n_step in range(len(alpha)):
    # 初始化邻接矩阵（完全图）
    A = np.zeros((N, N))
    for i in range(m0):
        for k in range(i + 1, m0):
            A[i, k] = 1
    A = A + A.T  # 对称化矩阵

    m_links = 3  # 每个新节点添加的边数
    p_rls = np.zeros(N)  # 节点被偏好连接的累积概率

    # 动态添加节点
    for i in range(m0, N):
        for _ in range(m_links):
            n = np.random.rand()  # 生成随机数决定连接类型
            
            # 偏好连接 (比例 = alpha)
            if n > alpha[n_step]:  # 修改条件：随机数 > alpha 时执行随机连接
                # 随机连接：任意选择一个节点
                k = np.random.randint(N)
                while k == i:  # 避免自环
                    k = np.random.randint(N)
                A[i, k] = 1
                A[k, i] = 1
            else:
                # 偏好连接：基于度的概率选择
                degree = np.sum(A, axis=1)  # 计算节点度
                uniform = np.zeros(N)
                r = degree > 0
                uniform[r] = 3  # 基础权重
                connected_set = uniform + p_rls  # 组合偏好概率
                connected_set = connected_set / np.sum(connected_set)  # 归一化
                
                k = np.random.choice(N, size=1, p=connected_set)
                while k == i:  # 避免自环
                    k = np.random.choice(N, size=1, p=connected_set)
                A[i, k] = 1
                A[k, i] = 1
                p_rls[k] += 1  # 更新偏好概率

    # 保存结果
    save_as_mat(A, f"Adjacency_2000_{alpha[n_step]}.mat")